# VirtueRed API Usage Guide

## Setup
Before running this notebook, ensure you have the following:
- Jupyter Notebook installed
- `requests` library installed (`pip install requests`)
- Your custom LLM code file in the `client/models/` directory
- The client address (obtained by running `python client.py` in the client directory)


## 1. Import Libraries and Set Base URL

Run this cell to import the necessary libraries and set the base URL for the API.

In [1]:
import requests
import json

# Base URL for the API (don't modify)
baseAPI = "https://redteaming.virtueai.io"

## 2. Initialize Scan

This cell defines and runs the function to initialize a new scan. 
IMPORTANT: `Run this cell only ONCE per scan`.

Before running:
1. Replace 'your_scan_name' with a name for this scan
2. Replace 'your_model_file.py' with the name of your custom LLM code file
3. Replace 'your_client_address' with the address obtained from running the client

After running, save the returned `filename` for use in subsequent cells.

In [2]:
def initialize_scan():
    ScanUrl = baseAPI + "/scan_subcategory"
    
    data = {
        "userId": "test_user",  # Don't modify this
        "selectedDatasets": [
            {
                "name": "Fairness",
                "subcategories": ["Salary Prediction", "Crime Prediction"]
            }
        ],  # Don't modify this
        "scanName": "your_scan_name",  # Replace with your chosen scan name
        "modelName": "your_model_file.py",  # Replace with your model file name
        "clientAddress": "http://your_client_address:4410"  # Replace with your client address, don't modify the default port 4410
    }
    
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(ScanUrl, json=data, headers=headers)

    if response.status_code == 200:
        print("Scan initialized successfully!")
        filename = response.json().get('filename')
        print(f"Filename: {filename}")
        return filename
    else:
        print(f"Scan initialization failed: {response.status_code}")
        print(response.text)
        return None

# Run the initialization
filename = initialize_scan()
print("Save this filename to use in the status check and results retrieval.")

Scan initialized successfully!
Filename: your_scan_name_your_model_file_2024-10-01T02_23_50.238Z
Save this filename to use in the status check and results retrieval.


## 3. Check Scan Status

This cell runs the function to check the status of your scan.
You can run this cell multiple times to monitor the progress of your scan.

If you're running this in a new session, make sure to set the `filename` variable
to the value returned by the initialization step before running this cell.

In [3]:
def check_scan_status(filename):
    CheckScanUrl = baseAPI + '/api/getreport'
    
    data = {"filename": filename}
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(CheckScanUrl, json=data, headers=headers)

    if response.status_code == 200:
        report_data = response.json()
        print(f"Scanning progress: {report_data.get('scanning_progress')}%")
    elif response.status_code == 400:
        print("Error: Filename not provided")
    elif response.status_code == 404:
        print("Error: File not found")
    else:
        print(f"Status check failed: {response.status_code}")
        print(response.text)

# Run the status check
# If 'filename' is not defined, replace it with the actual filename from the initialization step
check_scan_status(filename)


Scanning progress: 1.0%


## 4. Check Results and Save Data

This cell retrieves the current results of your scan, displays a summary, and saves the complete data to a JSON file. You can run this cell multiple times to see updated results, even if the scan is still in progress.

Before running this cell:
1. Ensure the `filename` variable is set to the correct value from the initialization step.
2. Set the `save_path` variable to your desired file path and name for saving the JSON data.

Usage notes:
- The cell will display a summary of the results in the notebook output.
- If results are successfully retrieved, they will be saved to the JSON file specified by `save_path`.
- Each run overwrites the previous JSON file with the latest data unless you change the `save_path`.
- The `json_data` variable contains all information returned by the API and can be used for further processing.


In [4]:
import json

def check_results(filename):
    resultUrl = baseAPI + f"/api/data/{filename}"
    
    response = requests.get(resultUrl)

    if response.status_code == 200:
        json_data = response.json()
        print("Run data summary:")
        print(f"Category scores: {json_data.get('averages')}")
        print(f"Subcategory scores: {json_data.get('averages_sub')}")
        print(f"Number of failure cases: {len(json_data.get('failures', []))}")
        print("Use json_data['scores'] for a detailed breakdown of results by risk levels.")
        print("Use json_data['failures'] to see the entire list of failure cases.")
        return json_data
    elif response.status_code == 404:
        print("Error: Run not found")
    else:
        print(f"Results retrieval failed: {response.status_code}")
        print(response.text)
    return None

def save_json_data(data, save_path):
    with open(save_path, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Data saved to {save_path}")

# Set the save path for the JSON file
# Replace this with your desired file path and name
save_path = "results.json"

# Run the results check
# If 'filename' is not defined, replace it with the actual filename from the initialization step
json_data = check_results(filename)

# Save the data if it was successfully retrieved
if json_data:
    save_json_data(json_data, save_path)

Error: Run not found


## Usage Notes

1. Always run the first cell to import libraries and set the base URL.
2. Run the second cell (Initialize Scan) `only once per scan`. Save the returned filename.
3. You can run the third cell (Check Scan Status) multiple times to monitor progress.
4. You can run the fourth cell (Check Results) multiple times to see current results.
5. If you close and reopen the notebook, you'll need to:
   - Run the first cell again
   - Manually set the `filename` variable to your saved value before running cells 3 and 4

Remember: The scan continues running on the server even if you close this notebook. You can always come back later to check status and results using the saved filename.